In [3]:
-- 1. Liệt kê thông tin ID, Họ tên, Tuổi của nhân viên.
SELECT	EmployeeID,
		CONCAT(FirstName,'',LastName) AS 'Full Name',
		(YEAR(GETDATE())-YEAR(BirthDate)) AS 'Year Old'
FROM Employees
GO

-- 2. Bổ sung thêm thông tin về quốc gia. Lọc lấy các khách hàng thuộc nước Pháp và Tây Ban Nha cùng mã số hóa đơn khách đã mua trong quý 3 và 4 của năm 1997.
SELECT O.OrderID, O.OrderDate, C.* 
FROM Customers AS C LEFT JOIN Orders AS O ON C.CustomerID = O.CustomerID
WHERE	(C.Country = 'France' OR C.Country = 'Spain') AND
		YEAR(O.OrderDate) = 1997 AND
		(DATEPART(QUARTER,OrderDate) = 3 OR DATEPART(QUARTER,OrderDate) = 4)
GO

-- 3. Giới hạn lại trong các hóa đơn khách đã mua trong quý 3 và 4 của năm 1997.
SELECT O.OrderID, O.OrderDate, C.* 
FROM Customers AS C LEFT JOIN Orders AS O ON C.CustomerID = O.CustomerID
WHERE	YEAR(O.OrderDate) = 1997 AND
		(DATEPART(QUARTER,OrderDate) = 3 OR DATEPART(QUARTER,OrderDate) = 4)
GO

-- 4. Tính tổng cước phí mỗi khách hàng đã trả cho mỗi nhà chuyên chở. Cước phí: Freight, Nhà chuyên chở: ShipVia/Shipper.
SELECT	S.CompanyName,
		ROUND(SUM(O.Freight),2) AS 'Total Freight'
FROM Orders AS O	LEFT JOIN Shippers AS S ON O.ShipVia = S.ShipperID
GROUP BY S.CompanyName
GO

-- 5. Tính tổng số tiền mỗi khách đã mua trong năm 1997.
SELECT	O.CustomerID,
		C.CompanyName,
		C.ContactName,
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Buy'
FROM Orders AS O	LEFT JOIN [Order Details] AS OD ON O.OrderID=OD.OrderID
					LEFT JOIN Customers AS C ON O.CustomerID = C.CustomerID
WHERE YEAR(O.OrderDate) = 1997
GROUP BY O.CustomerID,C.CompanyName,C.ContactName
GO

-- 6. Tính tổng doanh số mỗi nhân viên đã bán được trong năm 1997
SELECT	E.EmployeeID,
		CONCAT(E.FirstName,'',E.LastName) AS 'Full Name',
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Sell'
FROM Orders AS O	LEFT JOIN [Order Details] AS OD ON O.OrderID=OD.OrderID
					LEFT JOIN Employees AS E ON  O.EmployeeID = E.EmployeeID
WHERE YEAR(O.OrderDate) = 1997
GROUP BY E.EmployeeID, CONCAT(E.FirstName,'',E.LastName)
ORDER BY E.EmployeeID ASC
GO

-- 7. Tính tổng tiền bán được của mỗi loại sản phẩm trong năm 1997? 
SELECT	P.ProductID,
		P.ProductName,
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Sell'
FROM [Order Details] AS OD	LEFT JOIN Products AS P ON OD.ProductID = P.ProductID
							LEFT JOIN Orders AS O ON OD.OrderID=O.OrderID
WHERE YEAR(O.OrderDate) = 1997
GROUP BY P.ProductID, P.ProductName
ORDER BY P.ProductID ASC
GO

-- Từng quý năm 1997?
SELECT	P.ProductID,
		P.ProductName,
		(DATEPART(QUARTER,O.OrderDate)) AS 'Quater',
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Sell'
FROM [Order Details] AS OD	LEFT JOIN Products AS P ON OD.ProductID = P.ProductID
							LEFT JOIN Orders AS O ON OD.OrderID=O.OrderID
WHERE YEAR(O.OrderDate) = 1997
GROUP BY P.ProductID, P.ProductName,(DATEPART(QUARTER,O.OrderDate))
ORDER BY P.ProductID ASC
GO

-- 8. Liệt kê các quốc gia có trên 3 khách hàng
SELECT	C.Country,
		COUNT(C.Country) AS 'Count'
FROM Customers AS C
GROUP BY C.Country
HAVING COUNT(C.Country) > 3

-- 9. Liệt kê loại sản phẩm có số lượng sản phẩm nhiều nhất
SELECT	P.ProductID,
		P.ProductName,
		SUM(OD.Quantity) AS 'Amount'
FROM [Order Details] AS OD LEFT JOIN Products AS P ON OD.ProductID = P.ProductID
GROUP BY P.ProductID, P.ProductName
ORDER BY SUM(OD.Quantity) DESC
GO

-- 10. Liệt kê tên khách hàng mua hàng nhiều nhất trong năm 1997? 
SELECT TOP(10) 
		C.CustomerID,
		C.CompanyName,
		C.ContactName,
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Buy'
FROM Orders AS O	LEFT JOIN Customers AS C ON O.CustomerID = C.CustomerID
					LEFT JOIN [Order Details] AS OD ON O.OrderID = OD.OrderID
WHERE YEAR(O.OrderDate) = 1997
GROUP BY C.CustomerID, C.CompanyName, C.ContactName
ORDER BY ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) DESC
GO
-- Trong mỗi năm?

 SELECT * FROM (
SELECT	H.*,
		RANK() OVER (PARTITION BY Years ORDER BY [Total Money Buy] DESC) AS BXH 
FROM (
SELECT TOP(20) 
		C.CustomerID,
		C.CompanyName,
		C.ContactName,
		YEAR(O.OrderDate) AS 'Years',
		ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) AS 'Total Money Buy'
FROM Orders AS O	LEFT JOIN Customers AS C ON O.CustomerID = C.CustomerID
					LEFT JOIN [Order Details] AS OD ON O.OrderID = OD.OrderID

GROUP BY C.CustomerID, C.CompanyName, C.ContactName,YEAR(O.OrderDate)
ORDER BY ROUND(SUM(OD.UnitPrice * OD.Quantity*(1-OD.Discount)),2) DESC
) AS H 
) AS B 
WHERE B.BXH=1
GO

(9 rows affected)

EmployeeID,Full Name,Year Old
1,NancyDavolio,72
2,AndrewFuller,68
3,JanetLeverling,57
4,MargaretPeacock,83
5,StevenBuchanan,65
6,MichaelSuyama,57
7,RobertKing,60
8,LauraCallahan,62
9,AnneDodsworth,54


(22 rows affected)

(223 rows affected)

(3 rows affected)

...

OrderID,OrderDate,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
10609,1997-07-24 00:00:00.000,DUMON,Du monde entier,Janine Labrune,Owner,"67, rue des Cinquante Otages",Nantes,NULL,44000,France,40.67.88.88,40.67.89.89
10610,1997-07-25 00:00:00.000,LAMAI,La maison d'Asie,Annette Roulet,Sales Manager,1 rue Alsace-Lorraine,Toulouse,NULL,31000,France,61.77.61.10,61.77.61.11
10628,1997-08-12 00:00:00.000,BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,NULL,67000,France,88.60.15.31,88.60.15.32
10629,1997-08-12 00:00:00.000,GODOS,Godos Cocina Típica,José Pedro Freyre,Sales Manager,"C/ Romero, 33",Sevilla,NULL,41101,Spain,(95) 555 82 82,NULL
10631,1997-08-14 00:00:00.000,LAMAI,La maison d'Asie,Annette Roulet,Sales Manager,1 rue Alsace-Lorraine,Toulouse,NULL,31000,France,61.77.61.10,61.77.61.11
10634,1997-08-15 00:00:00.000,FOLIG,Folies gourmandes,Martine Rancé,Assistant Sales Agent,"184, chaussée de Tournai",Lille,NULL,59000,France,20.16.10.16,20.16.10.17
10663,1997-09-10 00:00:00.000,BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,NULL,13008,France,91.24.45.40,91.24.45.41
10671,1997-09-17 00:00:00.000,FRANR,France restauration,Carine Schmitt,Marketing Manager,"54, rue Royale",Nantes,NULL,44000,France,40.32.21.21,40.32.21.20
10679,1997-09-23 00:00:00.000,BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,NULL,67000,France,88.60.15.31,88.60.15.32
10683,1997-09-26 00:00:00.000,DUMON,Du monde entier,Janine Labrune,Owner,"67, rue des Cinquante Otages",Nantes,NULL,44000,France,40.67.88.88,40.67.89.89


OrderID,OrderDate,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
10643,1997-08-25 00:00:00.000,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545
10692,1997-10-03 00:00:00.000,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545
10702,1997-10-13 00:00:00.000,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545
10759,1997-11-28 00:00:00.000,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NULL,05021,Mexico,(5) 555-4729,(5) 555-3745
10625,1997-08-08 00:00:00.000,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NULL,05021,Mexico,(5) 555-4729,(5) 555-3745
10677,1997-09-22 00:00:00.000,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NULL,05023,Mexico,(5) 555-3932,NULL
10682,1997-09-25 00:00:00.000,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NULL,05023,Mexico,(5) 555-3932,NULL
10707,1997-10-16 00:00:00.000,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NULL,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
10741,1997-11-14 00:00:00.000,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NULL,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
10743,1997-11-17 00:00:00.000,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NULL,WA1 1DP,UK,(171) 555-7788,(171) 555-6750


CompanyName,Total Freight
QUICK,QUICK-Stop
SAVEA,Save-a-lot Markets
ERNSH,Ernst Handel
MEREP,Mère Paillarde
HUNGO,Hungry Owl All-Night Grocers
RATTC,Rattlesnake Canyon Grocery
SIMOB,Simons bistro
BERGS,Berglunds snabbköp
HILAA,HILARION-Abastos
FOLKO,Folk och fä HB


ProductID,ProductName,Amount
60,Camembert Pierrot,1577
59,Raclette Courdavault,1496
31,Gorgonzola Telino,1397
56,Gnocchi di nonna Alice,1263
16,Pavlova,1158
75,Rhönbräu Klosterbier,1155
24,Guaraná Fantástica,1125
40,Boston Crab Meat,1103
62,Tarte au sucre,1083
2,Chang,1057


CustomerID,CompanyName,ContactName,Years,Total Money Buy,BXH
ERNSH,Ernst Handel,Roland Mendel,1996,15568.06,1
QUICK,QUICK-Stop,Horst Kloss,1997,61109.91,1
ERNSH,Ernst Handel,Roland Mendel,1998,41210.65,1
